In [3]:
import requests
import simplejson
import unidecode
import operator
import json
import sys
from IPython.core.display import display, HTML
import pandas as pd
from collections import OrderedDict
from datetime import date

In [67]:

##########################################################################################
# PRODUCTO A BUSCAR

product = "vino destapador"
product_to_avoid = "xxx"

min_price_list = (1,501)#,3000,8000,16000)
max_price_list = (500,1000)#,3000,8000,16000,240000)
# TUS BARRIOS SEPARADOS POR ESPACIO
other_locations = "ezpeleta"
# SI EL MAXIMO VENDIDO ES 1000, y % 0.7 SOLO MUESTRA LOS QUE VENDIERON ARRIBA DE 700 
percentage_max_sell = "0.05"
# USED o NEW
sell_condition = "new" #"old"
# MINIMUM SELL FOR LOCATION
min_sell = 1

file = open("mla_results_" + product + ".csv","a+")

#########################################################################################

num = 0
for price in min_price_list:
    min_price = min_price_list[num]
    max_price = max_price_list[num]
    mla_list = search_mla_return_list(product,min_price,max_price)
    #return_tuples_from_list(mla_list,product,min_price,max_price)
    max_sold = return_max_sold_from_list(mla_list,product,min_price,max_price)
    #display(HTML('<style>        table {      border-collapse: collapse;   }  table, td, th {   border: 1px solid black;            }  </style>       '))
    ranger = '$' + str(min_price) + '-$' + str(max_price)
    #display(HTML("text/html", "<h2 style='padding: 10px'>Arc</h2><table class='table table-striped'> <thead>"))
    #display(HTML('<h3><font color=green>A#######BEGIN ANY LOCATION#######PRICE RANGE ' + ranger + '#######FOUND MAX UNITS SOLD' + str(max_sold) + '#############</h3></font>'))
    #print_products_from_list(mla_list,product,sell_condition,percentage_max_sell,max_sold,min_price,max_price)
    df = return_products_from_list_pandas(mla_list,product,sell_condition,percentage_max_sell,max_sold,min_price,max_price)
    #list(df)
    display(HTML(df.loc[:5, ['title', 'price','sold_quantity','state_name','city_name']].to_html()))
    df2 = return_products_from_list_in_locations_pandas(mla_list,product,sell_condition,other_locations,0 , int(min_price), int(max_price))
    #print ("DF2: ",list(df2))
    display(HTML(df2.loc[:5, ['title', 'price','sold_quantity','state_name','city_name']].to_html()))
    num = num + 1


Total(limited to 1000 with no token access): 161  start: 0  end  50


,title,price,sold_quantity,state_name,city_name
0,destapador y sacacorchos forma botella de vino...,54.99,500,Buenos Aires,LOMAS DE ZAMORA
3,destapador y sacacorcho imantado botella de vi...,57.00,250,Capital Federal,Villa Crespo
5,destapador sacacorcho botella vino cumple 40 ...,59.00,250,Buenos Aires,LOMAS DE ZAMORA


,title,price,sold_quantity,state_name,city_name
0,destapador y sacacorchos forma botella de vino...,54.99,500,Buenos Aires,LOMAS DE ZAMORA
3,destapador y sacacorcho imantado botella de vi...,57.00,250,Capital Federal,Villa Crespo
5,destapador sacacorcho botella vino cumple 40 ...,59.00,250,Buenos Aires,LOMAS DE ZAMORA


Total(limited to 1000 with no token access): 73  start: 0  end  50


,title,price,sold_quantity,state_name,city_name
2,sacacorchos automatico destapador de vinos abr...,728.00,500,Capital Federal,San Nicolás
17,sacacorchos automatico a pilas abridor de vino...,520.00,250,Capital Federal,Núñez
3,sacacorcho saca corchos electrico abridor vino...,907.65,250,Capital Federal,Villa Urquiza
1,sacacorchos automatico de acero abridor de vin...,999.00,250,Capital Federal,San Nicolás
0,sacacorchos automatico electrico abridor vinos...,885.51,150,Capital Federal,San Nicolás
7,sacacorcho destapador de vinos premium a palan...,749.00,50,Capital Federal,Belgrano
5,"souvenir destapador y sacacorcho, botella vino...",750.00,25,Capital Federal,Almagro o Villa Ortuzar


,title,price,sold_quantity,state_name,city_name
2,sacacorchos automatico destapador de vinos abr...,728.00,500,Capital Federal,San Nicolás
17,sacacorchos automatico a pilas abridor de vino...,520.00,250,Capital Federal,Núñez
3,sacacorcho saca corchos electrico abridor vino...,907.65,250,Capital Federal,Villa Urquiza
1,sacacorchos automatico de acero abridor de vin...,999.00,250,Capital Federal,San Nicolás
0,sacacorchos automatico electrico abridor vinos...,885.51,150,Capital Federal,San Nicolás
7,sacacorcho destapador de vinos premium a palan...,749.00,50,Capital Federal,Belgrano
5,"souvenir destapador y sacacorcho, botella vino...",750.00,25,Capital Federal,Almagro o Villa Ortuzar


In [39]:
df_resu = pd.DataFrame()

df_temp = df.loc[:, ['title', 'price','sold_quantity','state_name','city_name']].sort_values('sold_quantity',ascending=False)
#df_temp[:20]
df_temp2 = df_temp

df_temp2['title'] = df_temp2['title'].str.lower()

for palabra in product.split(' '):
    df_temp2 = df_temp2[ df_temp2['title'].str.contains(palabra)]
    df_resu = df_resu.append(df_temp2)
    #print ("R:",df_resu,"T:",df_temp2)

df_temp2    

,title,price,sold_quantity
2,sacacorchos automatico destapador de vinos abr...,728.00,500
3,sacacorcho saca corchos electrico abridor vino...,907.65,250
1,sacacorchos automatico de acero abridor de vin...,999.00,250
17,sacacorchos automatico a pilas abridor de vino...,520.00,250
0,sacacorchos automatico electrico abridor vinos...,885.51,150
7,sacacorcho destapador de vinos premium a palan...,749.00,50
5,"souvenir destapador y sacacorcho, botella vino...",750.00,25
8,sacacorcho destapador de vinos premium a palan...,749.00,25
48,"sacacorchos automático de acero con luz, desta...",950.00,5
10,sacacorcho destapador de vinos premium a palan...,650.00,5


In [66]:

def check_ALL_list_in_title(list,string ):
# ALL NEEDS TO MATCH
    result = 1
    for item in list:
        #print ("item:",item,"str:",string)
        if item.lower() in string.lower():
            result = 1
        else:
            result = 0
    return result


def check_ANY_list_in_title(list,string ):
# ANY NEEDS TO MATCH
    result = 0
    for item in list:
        #print ("item:",item,"str:",string)
        if item.lower() in string.lower() :
            result = 1
    return result



def search_mla_return_list(product,min_price,max_price):
    #print ("Returning complete MLA list for product",product)
    link = 'https://api.mercadolibre.com/sites/MLA/search?q=\'' + product + '\'' 
    price_filter = "&price=" + str(min_price) + "-" + str(max_price)
    link2 = link + '&offset=' + str(0) + '&limit=' + str(50) + price_filter
    r = requests.get(link2)
    #print (link2)
    c = r.json()
    #print (c['results'][0])
    c_orig = c
    c_partial = c['results']
    start =  c['paging']['offset']
    end =  c['paging']['limit']
    total =  c['paging']['total'] 
    total_saved = total
    total_c = c['results']
    maximum_sell = 0
    print ("Total(limited to 1000 with no token access):",total," start:",start," end ",end)
    if total > 100:
        total = 100
    # TEMOPORARY
    num_new = 0
    num=0
    num_new = 0
    while ( total > 51 and end > 10 ): # and 0):  ## <--- full list instead of first 50
        start = start + end
        if ( (c['paging']['total'] - start) < end  ):
            end = c['paging']['total'] - 1 - start 
        link2 = link + '&offset=' + str(start) + '&limit=' + str(end) + price_filter
        #print ("requesting LINK:",link2)
        r = requests.get(link2)
        c = r.json()
        c_partial = 0
        #print ("Total:",c['paging']['total']," start:",start," limit ",end)
        total_c = total_c + c['results']
        total = total - end 
        num = num + 1 
    return total_c

def return_max_sold_from_list(list,product,min_price,max_price):
    #print ("----- Searching max sold...");
    product_list = product.split()
    maximum_sell = 0
    num_new = 0
    for x in list[:]:
        #print(c['site_id'], c['results']['title'],c['results']['sold_quantity'], "$",c['results']['price'],c['site_id'], c['results']['address']['city_name'])
        sold_qty = list[num_new]['sold_quantity']
        price = list[num_new]['price']
        #print ("sold qty",sold_qty,"num:",num)
        title = unidecode.unidecode(list[num_new]['title']).lower()
        if ( sold_qty > maximum_sell and check_ALL_list_in_title(product_list,title ) and not check_ALL_list_in_title(product_to_avoid.split(),title ) and ( int(price) >= int(min_price) and int(price) <= int(max_price) )):
            maximum_sell = sold_qty
            #print ("new max sell found",maximum_sell)
        num_new = num_new + 1
    #print ("....max units sold at this range: ",maximum_sell)
    return maximum_sell

def return_tuples_from_list(list,product,min_price,max_price):
    print ("----- Searching maximum units sold...")
    product_list = product.split()
    maximum_sell = 0
    num_new = 0
    for x in list[:]:
        #print(c['site_id'], c['results']['title'],c['results']['sold_quantity'], "$",c['results']['price'],c['site_id'], c['results']['address']['city_name'])
        sold_qty = list[num_new]['sold_quantity']
        price = list[num_new]['price']
        title = list[num_new]['title']
        title_list = title.split()
        title_short = title_list[0:3]
        #print ("Title short:",title_short)
        #print ("sold qty",sold_qty,"num:",num)
        title = unidecode.unidecode(list[num_new]['title']).lower()
        if ( sold_qty > maximum_sell and check_ALL_list_in_title(product_list,title ) and not check_ALL_list_in_title(product_to_avoid.split(),title ) and ( int(price) >= int(min_price) and int(price) <= int(max_price) )):
            maximum_sell = sold_qty
            #print ("Title short:",title_short) 
            #print ("new max sell found",maximum_sell)
        num_new = num_new + 1
    return maximum_sell

       
def return_products_from_list_pandas(total_c,product,condition,percentage_max_sell,max_sell,min_price,max_price):
    from operator import itemgetter
    #print ("----- Printing products from list..max units sold:",max_sell," Percentage filter at:",percentage_max_sell,"...min:",min_price," max_price:",max_price)
    num = 0
    list_new = list()
    product_list = product.split()
    df = pd.DataFrame.from_dict(total_c)
    df_resu = pd.DataFrame()
    df_temp = df.loc[:, ['title', 'price','sold_quantity','address']].sort_values('sold_quantity',ascending=False)
    #df_temp[:20]
    df_temp['city_name'] = df_temp['address'].apply(lambda x: x.get('city_name'))
    df_temp['state_name'] = df_temp['address'].apply(lambda x: x.get('state_name'))
    df_temp2 = df_temp
    df_temp2['title'] = df_temp2['title'].str.lower()

    for palabra in product.split(' '):
        df_temp2 = df_temp2[ df_temp2['title'].str.contains(palabra)]
        df_resu = df_resu.append(df_temp2)
    #print ("R:",df_resu,"T:",df_temp2)
    return df_temp2
        

def return_products_from_list_in_locations_pandas(total_c,product,sell_condition,locations,min_sell,min_price,max_price):
    #print ("----- Printing products from list in LOCATION..min $:",min_price," max $:",max_price)
    #sorted(total_c, key=lambda sold_quantity: sold_quantity[8])
    #sorted(total_c, key=operator.itemgetter(16) )
    df = pd.DataFrame.from_dict(total_c)
    df_resu = pd.DataFrame()
    df_temp = df.loc[:, ['title', 'price','sold_quantity','address']].sort_values('sold_quantity',ascending=False)
    #df_temp[:20]
    df_temp['city_name'] = df_temp['address'].apply(lambda x: x.get('city_name'))
    df_temp['state_name'] = df_temp['address'].apply(lambda x: x.get('state_name'))
    df_temp2 = df_temp
    df_temp2['title'] = df_temp2['title'].str.lower()

    for palabra in product.split(' '):
        df_temp2 = df_temp2[ df_temp2['title'].str.contains(palabra)]
        df_resu = df_resu.append(df_temp2)
    #print ("R:",df_resu,"T:",df_temp2)
    return df_temp2
    
